# => `env`: `Jupyter`

# Procura Arquivos `Jupyter Notebook` no disco `C:\` e Grava essa Lista em uma página HTML contendo os seguintes itens:

1. Nome do Arquivo;
2. As primeiras linhas do arquivo *(restrito aos 1000 primeiros caracteres)*;
3. Pasta onde o arquivo está.

Tanto o **Nome do Arquivo**, quanto o nome da **Pasta** possuem links para os respectivos itens.

## Fases da Execução do App:

1. Procurar os arquivos e guardar na lista `files`;
2. Guardar essa lista no arquivo `notebooks.txt`;
3. Separar dessa lista, os arquivos *indesejáveis* (que estão em pastas de backup, bibliotecas do Python, Lixeira, etc);
   1. A variável `resultado_ok` guarda os *desejáveis*;
   2. A variável `resultado_indesejado` guarda os *indesejáveis*;
4. Criar o arquivo HTML com a listagem dos arquivos *desejáveis*: `notebooks.html`.

## 1:

In [1]:
from tqdm import tqdm
import glob
from bs4 import BeautifulSoup
import json


In [ ]:
# Procurar, recursivamente, todos os arquivos com sufixo 'ipynb': 
# Trouxe 572 resultados!

print("Procurando...")
files = glob.glob('C:/**/*.ipynb', recursive = True)
print(f'Foram encontrados {len(files)} arquivos Jupyter Notebook.')

> ***Observação:***
> 
> Tentei localizar os arquivos usando o método abaixo, mas ele só localizou **105 resultados**, contra **572 resultados** usando o método anterior:
>
> ```python
> for filename in glob.iglob('C:/**/*.ipynb', recursive = True):
>    print(filename)
>
> ```

# 2:

In [16]:
# Salvando o resultado em um arquivo para uso futuro:
for file in files:
    
    with open('notebooks.txt', 'a', encoding='utf-8') as f:
        f.write(file + '\n')

### Caso queira rodar o App com os dados já buscados anteriormente, pode rodar o código abaixo:

In [2]:
try:
    files
except:
    # Recuperando os dados arquivados:
    with open('notebooks.txt', 'r', encoding='utf-8') as f:
        l_files = f.readlines()
        
    for i, valor in enumerate(l_files):
        l_files[i] = valor[:-1]

    files = l_files

## 3:

Retirando da lista os arquivos indesejáveis.

Arquivos que estão dentro das seguintes pastas:
1. `\Cache\virtualenvs\`
2. `\Lib\`
3. `.ipynb_checkpoints`
4. `/$Recycle.Bin`
5. `\AppData\`


In [3]:
resultado_ok = [] # Resultado contendo o filtro
resultado_indesejado = [] # Resultado filtrado

indesejaveis = ['recycle.bin', 'cache', 'lib', '.ipynb_checkpoints', 'appdata'] # Pastas que não devem constar da listagem de ipynb

for indice, caminho in enumerate(tqdm(files)): # Loop na lista de arquivos
    copia = True
    for ind in indesejaveis: # loop na lista de indesejáveis
        # print(ind)
        for pedaco in caminho.split('\\'): # loop nas partes de cada arquivo
            # print(f'Processando: {pedaco} - {caminho[20:]}. [Índice: {indice}]')
            if ind in pedaco.lower():
                copia = False
                # if ind!='recycle.bin':
                #     print(f'pedaço:{pedaco} =>{caminho}')
                break
        if not copia:
            break
    if copia:
        resultado_ok.append(caminho)
    else:
        resultado_indesejado.append(caminho)

print(f'Total: {len(files)} - resultado_indesejado: {len(resultado_indesejado)} - resultado_ok: {len(resultado_ok)}')

100%|██████████| 572/572 [00:00<00:00, 22204.82it/s]

Total: 572 - resultado_indesejado: 177 - resultado_ok: 395


## 4:

### Função pra buscar o resumo do conteúdo do arquivo:


In [113]:

def pega_resumo(file, encoding='utf-8'):
    """
    Entrada: 
        str - Caminho completo do arquivo.
        str - Código de formatação. P.ex.: 'utf-8', 'ascii', etc. 
        Caso não seja informado, o padrão será 'utf-8'.

    Saída:
        resumo: str - Os 1.000 primeiros caracteres que estejam dentro dos conteúdos das células do arquivo 'ipynb'.
        ambiente: str - O nome contido na chave 'display_name' do arquivo.
        
    """
    try:
        with open(file, 'r', encoding=encoding) as f:
            texto = f.read()
        resumo = ''
        for i, c in enumerate(json.loads(texto)['cells']):
            if i>4:
                break
            if 'source' in c:
                for linha in c['source']:
                    resumo += linha + '\n'
        try:
            ambiente = json.loads(texto)['metadata']['kernelspec']['display_name']
        except:
            ambiente = 'N/A'
        
        return (resumo.replace('\n\n', '\n')[:1000].replace('\n', '<br>'), ambiente)
    except:
        return ('N/A', 'N/A')



### Criação e preenchimento da tabela `arquivos do usuário`:

In [114]:
# VERSÃO TESTE:
# 4.1
# Cria o html:
soup = BeautifulSoup("<html></html>")
original_tag = soup.html
original_tag.append(soup.new_tag('head'))
original_tag.head.append(soup.new_tag('meta'))
original_tag.head.meta['charset'] = 'utf-8'
original_tag.append(soup.new_tag('table'))
original_tag.table['border'] = '1'
## 4.2:
# Cria 1ª linha da tabela (cabeçalho):
linha_tag = soup.new_tag('tr')
original_tag.table.append(linha_tag)
# Preenche o cabeçalho da tabela:
cel_tag = soup.new_tag('th') #1
cel_tag.string = 'Arquivo'
linha_tag.append(cel_tag)
cel_tag = soup.new_tag('th') #2
cel_tag.string = 'Resumo'
linha_tag.append(cel_tag)
cel_tag = soup.new_tag('th') #3
cel_tag.string = 'Ambiente'
linha_tag.append(cel_tag)
cel_tag = soup.new_tag('th') #4
cel_tag.string = 'Pasta'
linha_tag.append(cel_tag)
for linha in range(len(resultado_ok)):
    
    arquivo = resultado_ok[linha].split('\\')[-1]
    pasta = resultado_ok[linha][:-len(arquivo)-1].replace('/', '\\')
    resumo_total_str, ambiente = pega_resumo(resultado_ok[linha])
    resumo_total = BeautifulSoup(resumo_total_str)
    if resumo_total.p==None:
        if resumo_total.body==None or resumo_total.body=='':
            resumo_tag = soup.new_tag('p')
            resumo_tag.string = '- X -'
        else:
            resumo_tag = resumo_total.body
    else:
        resumo_tag = resumo_total.p

    # Cria próxima linha da tabela:
    linha_tag = soup.new_tag('tr')
    original_tag.table.append(linha_tag)

    # Nova Linha >> Coluna 1 "Arquivo"
    cel_tag = soup.new_tag('td') # cria nova célula
    link_tag = soup.new_tag('a')
    link_tag['href'] = r'file://' + resultado_ok[linha].replace('/', '\\')
    link_tag.string = arquivo
    cel_tag.insert(1, link_tag)
    linha_tag.append(cel_tag)

    # Nova Linha >> # Coluna 2 (resumo):
    cel_tag = soup.new_tag('td') # cria nova célula
    cel_tag.append(resumo_tag) # tag contendo o resumo
    linha_tag.append(cel_tag)

    # Nova Linha >> Coluna 3 "Ambiente"
    cel_tag = soup.new_tag('td') # cria nova célula
    cel_tag.string = ambiente
    linha_tag.append(cel_tag)

    # Nova Linha >> Coluna 4 "Pasta"
    cel_tag = soup.new_tag('td') # cria nova célula
    link_tag = soup.new_tag('a')
    link_tag['href'] = r'file://' + pasta.replace('/', '\\')
    link_tag.string = pasta # <- variável "pasta"
    cel_tag.insert(1, link_tag)
    linha_tag.append(cel_tag)

# Salva em um arquivo html:
with open('notebooks.html', 'w', encoding='utf-8') as f:
    f.write(original_tag.prettify())
 # ACIMA ESTÁ OK!
 

150228

### Criação e preenchimento da tabela `arquivos do sistema`:

In [ ]:
# VERSÃO TESTE:
# 4.1
# Cria o html:
soup = BeautifulSoup("<html></html>")
original_tag = soup.html
original_tag.append(soup.new_tag('head'))
original_tag.head.append(soup.new_tag('meta'))
original_tag.head.meta['charset'] = 'utf-8'
original_tag.append(soup.new_tag('table'))
original_tag.table['border'] = '1'
## 4.2:
# Cria 1ª linha da tabela (cabeçalho):
linha_tag = soup.new_tag('tr')
original_tag.table.append(linha_tag)
# Preenche o cabeçalho da tabela:
cel_tag = soup.new_tag('th') #1
cel_tag.string = 'Arquivo'
linha_tag.append(cel_tag)
cel_tag = soup.new_tag('th') #2
cel_tag.string = 'Resumo'
linha_tag.append(cel_tag)
cel_tag = soup.new_tag('th') #3
cel_tag.string = 'Ambiente'
linha_tag.append(cel_tag)
cel_tag = soup.new_tag('th') #4
cel_tag.string = 'Pasta'
linha_tag.append(cel_tag)
for linha in range(len(resultado_indesejado)):
    
    arquivo = resultado_indesejado[linha].split('\\')[-1]
    pasta = resultado_indesejado[linha][:-len(arquivo)-1].replace('/', '\\')
    resumo_total_str, ambiente = pega_resumo(resultado_indesejado[linha])
    resumo_total = BeautifulSoup(resumo_total_str)
    if resumo_total.p==None:
        if resumo_total.body==None or resumo_total.body=='':
            resumo_tag = soup.new_tag('p')
            resumo_tag.string = '- X -'
        else:
            resumo_tag = resumo_total.body
    else:
        resumo_tag = resumo_total.p

    # Cria próxima linha da tabela:
    linha_tag = soup.new_tag('tr')
    original_tag.table.append(linha_tag)

    # Nova Linha >> Coluna 1 "Arquivo"
    cel_tag = soup.new_tag('td') # cria nova célula
    link_tag = soup.new_tag('a')
    link_tag['href'] = r'file://' + resultado_indesejado[linha].replace('/', '\\')
    link_tag.string = arquivo
    cel_tag.insert(1, link_tag)
    linha_tag.append(cel_tag)

    # Nova Linha >> # Coluna 2 (resumo):
    cel_tag = soup.new_tag('td') # cria nova célula
    cel_tag.append(resumo_tag) # tag contendo o resumo
    linha_tag.append(cel_tag)

    # Nova Linha >> Coluna 3 "Ambiente"
    cel_tag = soup.new_tag('td') # cria nova célula
    cel_tag.string = ambiente
    linha_tag.append(cel_tag)

    # Nova Linha >> Coluna 4 "Pasta"
    cel_tag = soup.new_tag('td') # cria nova célula
    link_tag = soup.new_tag('a')
    link_tag['href'] = r'file://' + pasta.replace('/', '\\')
    link_tag.string = pasta # <- variável "pasta"
    cel_tag.insert(1, link_tag)
    linha_tag.append(cel_tag)

# Salva em um arquivo html:
with open('notebooks_extra.html', 'w', encoding='utf-8') as f:
    f.write(original_tag.prettify())
 # ACIMA ESTÁ OK!
 

150228

In [72]:
# resultado_indesejado.pop(0)
resultado_indesejado[47]

'C:/Python39\\Lib\\site-packages\\nbformat\\tests\\test2.ipynb'

In [111]:
jt = '''{
    "metadata": {
        "name": "01 notebook"
    },
    "nbformat": 2,
    "worksheets": [
        {
            "cells": [
                {
     "cell_type": "markdown", 
     "source": [
      "# An introduction to the Jupyter notebook", 
      "", 
      "The IPython web notebook is a frontend that allows for new modes", 
      "of interaction with IPython: this web-based interface allows you to execute Python and IPython", 
      "commands in each input cell just like you would at the IPython terminal or Qt console, but you can", 
      "also save an entire session as a document in a file with the `.ipynb` extension.", 
      "", 
      "The document you are reading now is precisely an example of one such notebook, and we will show you", 
      "here how to best use this new interface.", 
      "", 
      "The first thing to understand is that a notebook consists of a sequence of 'cells' that can contain ", 
      "either text (such as this one) or code meant for execution (such as the next one):", 
      "", 
      "* Text cells can be written using [Markdown syntax](http://daringfireball.net/projects/markdown/syntax) ", 
      "(in a future release we will also provide support for reStructuredText and Sphinx integration, and we ", 
      "welcome help from interested contributors to make that happen).", 
      "", 
      "* Code cells take IPython input (i.e. Python code, `%magics`, `!system calls`, etc) like IPython at", 
      "the terminal or at the Qt Console.  The only difference is that in order to execute a cell, you *must*", 
      "use `Shift-Enter`, as pressing `Enter` will add a new line of text to the cell.  When you type ", 
      "`Shift-Enter`, the cell content is executed, output displayed and a new cell is created below.  Try", 
      "it now by putting your cursor on the next cell and typing `Shift-Enter`:"
     ]
    },


{
     "cell_type": "code", 
     "collapsed": false, 
     "input": [
      "\"This is the new Jupyter notebook\""
     ], 
     "language": "python", 
     "outputs": [
      {
       "output_type": "pyout", 
       "prompt_number": 1, 
       "text": [
        "'This is the new Jupyter notebook'"
       ]
      }
     ], 
     "prompt_number": 1
    }

            ]
        }
    ]
}'''

In [112]:
print(json.loads(jt))

JSONDecodeError: Expecting ',' delimiter: line 42 column 9 (char 1964)

---
# Rascunhos:

### Colocando imagem no link:

In [ ]:
# Abrindo o arquivo com o VSC:
link_tag = soup.new_tag('img')
link_tag['src'] = r'https://img.icons8.com/material-outlined/32/000000/code.png'
link_tag['href'] = r'http://blablabla.com/' 
# link_tag.string = 'VSC'
td_tag.insert(1, link_tag)    

#### Gabaritos de inclusão de células em `tabelas`:

1. O Conteúdo **é** um link:
   ```python
    cel_tag = soup.new_tag('td') # cria nova célula
    cel_tag.string = 'texto (além do link)' # Pode ser desnecessário!
    link_tag = soup.new_tag('a') # cria um link
    link_tag['href'] = r'http://www.blablabla.com/' # define a url do link
    link_tag.string = 'texto do link' # define o texto do link
    cel_tag.insert(1, link_tag) # insere o link na célula
    linha_tag.append(cel_tag) # insere a célula na linha
    
   ```


2. O Conteudo **não é** um link:
    ```python
    cel_tag = soup.new_tag('td') # cria nova célula
    cel_tag.string = 'texto da célula'
    linha_tag.append(cel_tag) # insere a célula na linha
   ```

## Uso de `powershell` com o `python`:

In [130]:
%pwd
# %run 'C:\Users\leite.aml\OneDrive - Polícia Federal\Programação\Python\Packt\Powershell.ipynb'.replace('\\', '/')

'c:\\Users\\leite.aml\\OneDrive - Polícia Federal\\Programação\\Python\\util'

In [137]:
%run ./Powershell.ipynb

SyntaxError: invalid syntax (Temp/ipykernel_21968/3179874894.py, line 1)

SyntaxError: invalid syntax (Temp/ipykernel_21968/3179874894.py, line 1)